## Agents Pipeline In Langchain

In [4]:
# Load The env Variables
import os
from dotenv import load_dotenv

load_dotenv()
os.environ['OPENAI_API_KEY']=os.getenv('OPENAI_API_KEY')
# print(os.getenv('OPENAI_API_KEY'))

In [36]:
# install required library 
# !pip install arxiv,wikipedia,langchainhub
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
# from langchain_community.tools import GoogleSearchRun
# from langchain_community.utilities import GoogleSearchAPIWrapper

In [6]:
# Crafting Wikipedia api search wrraper and query tool
wiki_apisearch_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki_tool=WikipediaQueryRun(api_wrapper=wiki_apisearch_wrapper)

In [7]:
# Retrieve Data from Web page to act as a datasource
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
data_source=WebBaseLoader("https://docs.smith.langchain.com/")
loaded_data_source=data_source.load()
splitted_data_source=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(loaded_data_source)
vectore_store_from_datasource=FAISS.from_documents(splitted_data_source,OpenAIEmbeddings())
vectore_store_retriever=vectore_store_from_datasource.as_retriever()
vectore_store_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002B4AB62B640>)

In [8]:
# Create customize tool bases on personal retriever to server on agents later
from langchain.tools.retriever import create_retriever_tool
langsmith_retriever_tool=create_retriever_tool(vectore_store_retriever,"langsmith_search",
                                     "Search for information about langsmith.For any question about Langsmith you must use this tool")
langsmith_retriever_tool.name

'langsmith_search'

In [9]:
# Create search Tool for arxiv platform
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_api_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv_search_tool=ArxivQueryRun(api_wrapper=arxiv_api_wrapper)
arxiv_search_tool.name

'arxiv'

In [11]:
# Combining All The previous Tools
combined_tools=[wiki_tool,langsmith_retriever_tool,arxiv_search_tool]
combined_tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\mehdi\\PycharmProjects\\Langchain-Ops\\lgops\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for information about langsmith.For any question about Langsmith you must use this tool', args_schema=<class 'langchain.tools.retriever.RetrieverInput'>, func=<bound method BaseRetriever.get_relevant_documents of VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002B4AB62B640>)>, coroutine=<bound method BaseRetriever.aget_relevant_documents of VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002B4AB62B640>)>),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptio

In [16]:
# Using Langchain Hub By Langsmith for Polished Prompts
from langchain import hub
# Pull the prompt to use with possible modification
agents_prompt_tempplate=hub.pull("hwchase17/openai-functions-agent")
agents_prompt_tempplate.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [33]:
# Create Agent for The previous comnined tools
from langchain.agents import create_openai_tools_agent
from langchain_openai import ChatOpenAI
openai_agent_tool=create_openai_tools_agent(llm=ChatOpenAI(model="gpt-3.5-turbo",temperature=0),tools=combined_tools,prompt=agents_prompt_tempplate)
openai_agent_tool

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='chat_history', opt

In [34]:
# Execute The Agents to Perform Operations
from langchain.agents import AgentExecutor
openai_agents_executer=AgentExecutor(agent=openai_agent_tool,tools=combined_tools,verbose=True)
openai_agents_executer.invoke({"input":"What is the latest version of java programming language ?"})

--- Logging error ---
Traceback (most recent call last):
  File "c:\Users\mehdi\PycharmProjects\Langchain-Ops\lgops\lib\site-packages\langchain_core\callbacks\manager.py", line 1985, in _configure
    handler = LangChainTracer(project_name=tracer_project)
  File "c:\Users\mehdi\PycharmProjects\Langchain-Ops\lgops\lib\site-packages\langchain_core\tracers\langchain.py", line 91, in __init__
    self.client = client or get_client()
  File "c:\Users\mehdi\PycharmProjects\Langchain-Ops\lgops\lib\site-packages\langchain_core\tracers\langchain.py", line 54, in get_client
    _CLIENT = Client()
  File "c:\Users\mehdi\PycharmProjects\Langchain-Ops\lgops\lib\site-packages\langsmith\client.py", line 378, in __init__
    _validate_api_key_if_hosted(self.api_url, self.api_key)
  File "c:\Users\mehdi\PycharmProjects\Langchain-Ops\lgops\lib\site-packages\langsmith\client.py", line 245, in _validate_api_key_if_hosted
    raise ls_utils.LangSmithUserError(
langsmith.utils.LangSmithUserError: API key mu



> Entering new AgentExecutor chain...


--- Logging error ---
Traceback (most recent call last):
  File "c:\Users\mehdi\PycharmProjects\Langchain-Ops\lgops\lib\site-packages\langchain_core\callbacks\manager.py", line 1985, in _configure
    handler = LangChainTracer(project_name=tracer_project)
  File "c:\Users\mehdi\PycharmProjects\Langchain-Ops\lgops\lib\site-packages\langchain_core\tracers\langchain.py", line 91, in __init__
    self.client = client or get_client()
  File "c:\Users\mehdi\PycharmProjects\Langchain-Ops\lgops\lib\site-packages\langchain_core\tracers\langchain.py", line 54, in get_client
    _CLIENT = Client()
  File "c:\Users\mehdi\PycharmProjects\Langchain-Ops\lgops\lib\site-packages\langsmith\client.py", line 378, in __init__
    _validate_api_key_if_hosted(self.api_url, self.api_key)
  File "c:\Users\mehdi\PycharmProjects\Langchain-Ops\lgops\lib\site-packages\langsmith\client.py", line 245, in _validate_api_key_if_hosted
    raise ls_utils.LangSmithUserError(
langsmith.utils.LangSmithUserError: API key mu


Invoking: `wikipedia` with `Java (programming language)`


Page: Java (programming language)
Summary: Java is a high-level, class-based, object-oriented programming language that is designed to have as few implementation dependencies as possible. It is a gene

--- Logging error ---
Traceback (most recent call last):
  File "c:\Users\mehdi\PycharmProjects\Langchain-Ops\lgops\lib\site-packages\langchain_core\callbacks\manager.py", line 1985, in _configure
    handler = LangChainTracer(project_name=tracer_project)
  File "c:\Users\mehdi\PycharmProjects\Langchain-Ops\lgops\lib\site-packages\langchain_core\tracers\langchain.py", line 91, in __init__
    self.client = client or get_client()
  File "c:\Users\mehdi\PycharmProjects\Langchain-Ops\lgops\lib\site-packages\langchain_core\tracers\langchain.py", line 54, in get_client
    _CLIENT = Client()
  File "c:\Users\mehdi\PycharmProjects\Langchain-Ops\lgops\lib\site-packages\langsmith\client.py", line 378, in __init__
    _validate_api_key_if_hosted(self.api_url, self.api_key)
  File "c:\Users\mehdi\PycharmProjects\Langchain-Ops\lgops\lib\site-packages\langsmith\client.py", line 245, in _validate_api_key_if_hosted
    raise ls_utils.LangSmithUserError(
langsmith.utils.LangSmithUserError: API key mu

The latest version of the Java programming language is Java 17, which was released on September 14, 2021.

> Finished chain.


--- Logging error ---
Traceback (most recent call last):
  File "c:\Users\mehdi\PycharmProjects\Langchain-Ops\lgops\lib\site-packages\langchain_core\callbacks\manager.py", line 1985, in _configure
    handler = LangChainTracer(project_name=tracer_project)
  File "c:\Users\mehdi\PycharmProjects\Langchain-Ops\lgops\lib\site-packages\langchain_core\tracers\langchain.py", line 91, in __init__
    self.client = client or get_client()
  File "c:\Users\mehdi\PycharmProjects\Langchain-Ops\lgops\lib\site-packages\langchain_core\tracers\langchain.py", line 54, in get_client
    _CLIENT = Client()
  File "c:\Users\mehdi\PycharmProjects\Langchain-Ops\lgops\lib\site-packages\langsmith\client.py", line 378, in __init__
    _validate_api_key_if_hosted(self.api_url, self.api_key)
  File "c:\Users\mehdi\PycharmProjects\Langchain-Ops\lgops\lib\site-packages\langsmith\client.py", line 245, in _validate_api_key_if_hosted
    raise ls_utils.LangSmithUserError(
langsmith.utils.LangSmithUserError: API key mu

{'input': 'What is the latest version of java programming language ?',
 'output': 'The latest version of the Java programming language is Java 17, which was released on September 14, 2021.'}

In [35]:
# Request Information about a research paper based on his id 
openai_agents_executer.invoke({"input":"Give me the idea of reasearch paper with this reference : 2310.05421 "})

--- Logging error ---
Traceback (most recent call last):
  File "c:\Users\mehdi\PycharmProjects\Langchain-Ops\lgops\lib\site-packages\langchain_core\callbacks\manager.py", line 1985, in _configure
    handler = LangChainTracer(project_name=tracer_project)
  File "c:\Users\mehdi\PycharmProjects\Langchain-Ops\lgops\lib\site-packages\langchain_core\tracers\langchain.py", line 91, in __init__
    self.client = client or get_client()
  File "c:\Users\mehdi\PycharmProjects\Langchain-Ops\lgops\lib\site-packages\langchain_core\tracers\langchain.py", line 54, in get_client
    _CLIENT = Client()
  File "c:\Users\mehdi\PycharmProjects\Langchain-Ops\lgops\lib\site-packages\langsmith\client.py", line 378, in __init__
    _validate_api_key_if_hosted(self.api_url, self.api_key)
  File "c:\Users\mehdi\PycharmProjects\Langchain-Ops\lgops\lib\site-packages\langsmith\client.py", line 245, in _validate_api_key_if_hosted
    raise ls_utils.LangSmithUserError(
langsmith.utils.LangSmithUserError: API key mu



> Entering new AgentExecutor chain...


--- Logging error ---
Traceback (most recent call last):
  File "c:\Users\mehdi\PycharmProjects\Langchain-Ops\lgops\lib\site-packages\langchain_core\callbacks\manager.py", line 1985, in _configure
    handler = LangChainTracer(project_name=tracer_project)
  File "c:\Users\mehdi\PycharmProjects\Langchain-Ops\lgops\lib\site-packages\langchain_core\tracers\langchain.py", line 91, in __init__
    self.client = client or get_client()
  File "c:\Users\mehdi\PycharmProjects\Langchain-Ops\lgops\lib\site-packages\langchain_core\tracers\langchain.py", line 54, in get_client
    _CLIENT = Client()
  File "c:\Users\mehdi\PycharmProjects\Langchain-Ops\lgops\lib\site-packages\langsmith\client.py", line 378, in __init__
    _validate_api_key_if_hosted(self.api_url, self.api_key)
  File "c:\Users\mehdi\PycharmProjects\Langchain-Ops\lgops\lib\site-packages\langsmith\client.py", line 245, in _validate_api_key_if_hosted
    raise ls_utils.LangSmithUserError(
langsmith.utils.LangSmithUserError: API key mu


Invoking: `arxiv` with `{'query': '2310.05421'}`


Published: 2023-10-09
Title: Automating Customer Service using LangChain: Building custom open-source GPT Chatbot for organizations
Authors: Keivalya Pandya, Mehfuza Holia
Summary: In the digital age,

--- Logging error ---
Traceback (most recent call last):
  File "c:\Users\mehdi\PycharmProjects\Langchain-Ops\lgops\lib\site-packages\langchain_core\callbacks\manager.py", line 1985, in _configure
    handler = LangChainTracer(project_name=tracer_project)
  File "c:\Users\mehdi\PycharmProjects\Langchain-Ops\lgops\lib\site-packages\langchain_core\tracers\langchain.py", line 91, in __init__
    self.client = client or get_client()
  File "c:\Users\mehdi\PycharmProjects\Langchain-Ops\lgops\lib\site-packages\langchain_core\tracers\langchain.py", line 54, in get_client
    _CLIENT = Client()
  File "c:\Users\mehdi\PycharmProjects\Langchain-Ops\lgops\lib\site-packages\langsmith\client.py", line 378, in __init__
    _validate_api_key_if_hosted(self.api_url, self.api_key)
  File "c:\Users\mehdi\PycharmProjects\Langchain-Ops\lgops\lib\site-packages\langsmith\client.py", line 245, in _validate_api_key_if_hosted
    raise ls_utils.LangSmithUserError(
langsmith.utils.LangSmithUserError: API key mu

Based on the reference 2310.05421, the research paper idea could be "Automating Customer Service using LangChain: Building custom open-source GPT Chatbot for organizations." This paper could focus on the development and implementation of a custom open-source GPT Chatbot, specifically designed for automating customer service processes in organizations. The research could explore the effectiveness, efficiency, and impact of using such a chatbot in enhancing customer service experiences.

> Finished chain.


--- Logging error ---
Traceback (most recent call last):
  File "c:\Users\mehdi\PycharmProjects\Langchain-Ops\lgops\lib\site-packages\langchain_core\callbacks\manager.py", line 1985, in _configure
    handler = LangChainTracer(project_name=tracer_project)
  File "c:\Users\mehdi\PycharmProjects\Langchain-Ops\lgops\lib\site-packages\langchain_core\tracers\langchain.py", line 91, in __init__
    self.client = client or get_client()
  File "c:\Users\mehdi\PycharmProjects\Langchain-Ops\lgops\lib\site-packages\langchain_core\tracers\langchain.py", line 54, in get_client
    _CLIENT = Client()
  File "c:\Users\mehdi\PycharmProjects\Langchain-Ops\lgops\lib\site-packages\langsmith\client.py", line 378, in __init__
    _validate_api_key_if_hosted(self.api_url, self.api_key)
  File "c:\Users\mehdi\PycharmProjects\Langchain-Ops\lgops\lib\site-packages\langsmith\client.py", line 245, in _validate_api_key_if_hosted
    raise ls_utils.LangSmithUserError(
langsmith.utils.LangSmithUserError: API key mu

{'input': 'Give me the idea of reasearch paper with this reference : 2310.05421 ',
 'output': 'Based on the reference 2310.05421, the research paper idea could be "Automating Customer Service using LangChain: Building custom open-source GPT Chatbot for organizations." This paper could focus on the development and implementation of a custom open-source GPT Chatbot, specifically designed for automating customer service processes in organizations. The research could explore the effectiveness, efficiency, and impact of using such a chatbot in enhancing customer service experiences.'}